In [1]:
import os
from openai import OpenAI
from langchain import hub
from pprint import pprint
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain_community.tools import ArxivQueryRun
from langchain.agents import create_openai_tools_agent
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.utilities import WikipediaAPIWrapper


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [2]:
def content_generation(SYSTEM_TEMPLATE, USER_TEMPLATE):
    response = client.chat.completions.create(
    model="gpt-4-0125-preview",
    messages=[
        {"role": "system", "content": SYSTEM_TEMPLATE},
        {"role": "user", "content": USER_TEMPLATE},
        ],
        temperature=0.8,
        response_format={'type':"json_object"}  
        )
    data = response.choices[0].message.content

    return data

In [3]:
DESCRIPTION_EXTRACTION_SYSTEM_TEMPLATE="Give me a long detailed description on the topic: {topic_name}"

In [4]:
TITLE_GENERATION_SYSTEM_TEMPLATE = "Craft a catchy title for a blog based on current trend by analyzing the given topic well: {topic_name}. Generate the response in JSON format provided as in the example."
TITLE_GENERATION_USER_TEMPLATE = "Example:\n{'blog_title': 'Write title here'}"

In [5]:
FORMAT_KEYWORD_GENERATION_SYSTEM_TEMPLATE = "You are responsible for generating a short format/structure for a blog by analyzing the given title well: {blog_title}. Also, generate SEO-friendly keywords for good ranking of the blog. Generate the response in JSON format provided as in the example."
FORMAT_KEYWORD_GENERATION_USER_TEMPLATE = "Example:\n{'format': 'Write format here',\n'keywords': ['Write SEO friendly keywords here']}"

In [6]:
BLOG_GENERATION_SYSTEM_TEMPLATE = "You are an expert blogger. You are having great creativity in writing quality blogs by covering all the aspects of the title based on given context \n<context>\n{context}\n</context>. Your writing focuses on providing valuable insights, actionable tips, or entertaining narratives that connect with your target audience. But you have some major flaws:\n<flaws>\n1. Making more grammar & typographical errors\n2. No proper structure & flow for the writings\n3. Getting diverted from the main purpose\n4. Forgetting about the target audience\n5. Less engagement & interactive elements\n6. Lack of clarity in explanation. \n</flaws> \nCreate a {tone_of_the_blog_professional_casual_formal_friendly} tone simple, understandable blog post about the title: {blog_title} in the {domain} industry by following the format: {format_of_the_blog} and emphasizing these SEO keywords: {keywords}. Write over {number_of_words} words. The blog post should be in a {type_of_blog_beginner_or_advanced} guide style with {grammar_error_in_major_minor_zero} grammar errors. Ensure in the 'content' you are including the entire summary which may contain a combination of any of these points: \n<schema>\n1. Introduction, \n2. Table of contents, \n3. Detailed description, \n4. Sub-headings, \n5. codes/scripts explanations, \n6. Conclusion, \n7. Thank you message, \n8. Resources(avoid giving links), \n9. Recap of the blog, etc.]. \n</schema>\nShow your creativity to write this blog beautifully. Ensure the blog provides valuable insights, engages the reader, and is well-structured with clear headings and subheadings. Incorporate relevant examples, statistics, and references to support key points. Maintain professionalism and authority while catering to the target audience's preferences. \n<guidelines>\n- Before generating the response must ensure that you are repeating your flaws to make this blog a more humanistic response\n- Use transition words.\n- Must include the code snippet with an explanation if the title contains any technical concepts. \n- Use active voice.\n- Please generate the response in JSON format.\n</guidelines>"
BLOG_GENERATION_USER_TEMPLATE = "\nExample json format:\n{'blog': {'title': 'write title here', 'content': 'writte content here', 'key_words': 'write key points here'}}"

In [7]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_contain_chars_max=500)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_contain_chars_max=500)
arxiv=ArxivQueryRun(arxiv_wrapper=arxiv_wrapper)

In [8]:
tools = [wiki, arxiv]

In [9]:
llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)
prompt=hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [10]:
def blog_content(topic_name, tone_of_the_blog_professional_casual_formal_friendly, domain, number_of_words, type_of_blog_beginner_or_advanced, grammar_error_in_major_minor_zero):
    response = agent_executor.invoke({'input': DESCRIPTION_EXTRACTION_SYSTEM_TEMPLATE.format(topic_name=topic_name)})
    context=response['output']
    res0=eval(content_generation(TITLE_GENERATION_SYSTEM_TEMPLATE.format(topic_name=topic_name), TITLE_GENERATION_USER_TEMPLATE))
    blog_title=res0['blog_title']
    res1 = eval(content_generation(FORMAT_KEYWORD_GENERATION_SYSTEM_TEMPLATE.format(blog_title=blog_title), FORMAT_KEYWORD_GENERATION_USER_TEMPLATE))
    format=res1['format']
    keywords=res1['keywords']
    res2 = content_generation(BLOG_GENERATION_SYSTEM_TEMPLATE.format(context=context,tone_of_the_blog_professional_casual_formal_friendly=tone_of_the_blog_professional_casual_formal_friendly, blog_title=blog_title, domain=domain, format_of_the_blog=format, keywords=keywords, number_of_words=number_of_words, type_of_blog_beginner_or_advanced=type_of_blog_beginner_or_advanced, grammar_error_in_major_minor_zero=grammar_error_in_major_minor_zero,), BLOG_GENERATION_USER_TEMPLATE)

    return res2

In [11]:
topic_name='Multi modal Language Model'
tone_of_the_blog_professional_casual_formal_friendly = 'casual'
domain = 'Data Science'
number_of_words = 1000
type_of_blog_beginner_or_advanced = 'advanced'
grammar_error_in_major_minor_zero = 'minor'

result = blog_content(topic_name=topic_name, tone_of_the_blog_professional_casual_formal_friendly=tone_of_the_blog_professional_casual_formal_friendly, domain=domain, number_of_words=number_of_words, type_of_blog_beginner_or_advanced=type_of_blog_beginner_or_advanced, grammar_error_in_major_minor_zero=grammar_error_in_major_minor_zero)

In [12]:
pprint(eval(result)['blog']['title'])

("Bridging Worlds: The Power of Multi-Modal Language Models in Today's Tech "
 'Landscape')


In [13]:
pprint(eval(result)['blog']['content'])

("Hey there, tech enthusiasts! Today we're diving into a game-changing area in "
 "the data science industry: multi-modal language models. These aren't your "
 "everyday AI systems; they're more like the Swiss Army knife in the tech "
 "toolbox. Let's break it down and see why they're making waves across "
 'sectors.\n'
 '\n'
 '**Introduction**\n'
 '\n'
 'In the digital age, where information zooms around in various formats—text, '
 'images, audio, and video—multi-modal language models stand out by '
 "understanding and processing this diverse data simultaneously. They're "
 "pivotal in today's tech landscape, pushing the boundaries of machine "
 'learning and AI capabilities. But what makes them so special? Hang tight, as '
 'we explore this fascinating world.\n'
 '\n'
 '**Understanding Multi-Modal Language Models**\n'
 '\n'
 'At their core, multi-modal language models are AI systems designed to digest '
 'and interpret data across different formats. Unlike traditional models that '
 